In [1]:
# load module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wntr
import wntr_utils
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import csv
import random

# Jupyter Notebook to generate random demand patterns and tank initial conditions for Minitorn


In [4]:
output_file = 'second_batch_three_year_demands_minitown.csv'
random_init_points = 'second_batch_random_init_points.csv'
random_initial_tanks = 'second_batch_minitown_tank_initial_conditions.csv'
all_data_path = '../Demand_patterns/data/allData.txt'

In [5]:
# load STREaM data (STREaM is a simulator of end-uses and water demand patterns)
# This dataset contains 3 years of hourly (26280 data points) synthetic demand data 
# for 4 user_classes = ['low','average','high','morning','night']. There are 500 user for each class,
# with the first 10 users having leaks. We need to remove these users for our purposes. 
df = pd.read_csv(all_data_path, header=None)
# 
# create headers
user_classes = ['low','average','high','morning','night']
headers = []
for user_class in user_classes:
    suffixes = (('_'+user_class+' ')*400).split() # original
    for suffix, number in zip(suffixes,range(1,401)):
        headers.append('user'+suffix+'_'+'%03d'%number)  
        
# reverse df and change df columns
df = df.T
df.columns = headers

# remove first 10 of each class (these have leaks, so not useful for our purposes)
users_to_remove = np.array(headers).reshape(-1,10)[::40].ravel().tolist()
df.drop(users_to_remove, axis = 1, inplace = True)
headers = df.columns

In [6]:
# this function creates a hourly pattern of multipliers (up to 26280 hours long) mixing together n (max 1950) patterns
# from the STREaM dataset; it scales within a range (as expected for WNTR/EPANET input)
def create_demand_pattern(df_patterns, n_hours=168, n_mix=50, bound = [0.3,0.9], noise_intensity=0.01):
    cols =df_patterns.columns.to_list()
    shuffle(cols)
    df_patterns = df_patterns[cols]
    s = MinMaxScaler(bound)
    temp=np.squeeze(s.fit_transform(df_patterns.iloc[:n_hours,:n_mix].mean(axis=1).values.reshape(-1,1)))
    dem_pat = temp + np.random.randn(n_hours)*noise_intensity
    return dem_pat

In [9]:
# load EPANET model
inp_file = 'minitown_map.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

# save original patterns
names, patterns =[],[]
for name, pat in wn.patterns():
    names.append(name)
    patterns.append(pat.multipliers)
df_pat_orig = pd.DataFrame(data = patterns).T
df_pat_orig.columns = names

C:\Users\Andres\anaconda3\lib\site-packages\wntr\epanet\io.py:2173: UserWarning: Not all curves were used in "minitown_map.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [10]:
# Generate a 3-year pattern for the District Metered Area (DMA)
names, patterns =[],[]
for name, pat in wn.patterns():
    names.append(name)
    temp = create_demand_pattern(df, n_hours=24*365*3, n_mix=150)
    pat.multipliers = temp
    patterns.append(pat.multipliers)
df_pat_new = pd.DataFrame(data = patterns).T
df_pat_new.columns = names
df_pat_new.to_csv(output_file)

In [11]:
# Create a pandas dataframe with columns = each DMA pattern and 38 random initial pattern values
df_pat_new = pd.read_csv(output_file)
values=[]
weekly_simulation_duration = 168
ten_days_simulatino_duration = 240
weekly=False

if weekly:
    limit = weekly_simulation_duration
else:
    limit = ten_days_simulatino_duration

for i in range(0,38):
    value = random.randint(0, len(df_pat_new)-limit)        
    values.append(value)
pattern_random_init_point = pd.DataFrame(data = values)
pattern_random_init_point.columns = ['start']
print(pattern_random_init_point)
pattern_random_init_point.to_csv(random_init_points)

    start
0    6060
1    6491
2   17176
3    7355
4    2145
5   24171
6   15602
7   23229
8   20317
9   23187
10  10731
11  25477
12  19504
13   4600
14  11212
15  15349
16  23572
17  14688
18  17251
19  19700
20  15704
21  17657
22  17680
23   9999
24  10621
25  25252
26   2460
27   6396
28  13197
29  17673
30  20859
31   3647
32  23095
33  14965
34  12010
35  15192
36   7480
37  16556


In [12]:
# Create a pandas dataframe with initial tank levels for the 7 C-Town tanks and 38 weeks
values=[]
#tank_names = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7']
tank_names = ['TANK']
for name in tank_names:
    aux=[]
    for i in range(0,38):
        tank_max_value = wn.get_node(name).max_level
        min_value = tank_max_value*0.10
        max_value = tank_max_value*0.80
        value = random.uniform(min_value, max_value)        
        aux.append(value)
    values.append(aux)
tank_initial_levels = pd.DataFrame(data = values).T
tank_initial_levels.columns = tank_names
print(tank_initial_levels)
tank_initial_levels.to_csv(random_initial_tanks)

        TANK
0   4.803270
1   4.850546
2   2.642841
3   4.779293
4   3.682787
5   3.486859
6   2.764002
7   3.002364
8   3.119228
9   4.094907
10  4.948651
11  1.069782
12  2.371399
13  0.936612
14  2.608917
15  5.116737
16  2.637573
17  0.872869
18  2.508483
19  2.925417
20  3.275713
21  4.508348
22  1.791803
23  5.002481
24  3.872937
25  3.988391
26  3.657865
27  1.797437
28  2.857792
29  4.676100
30  1.555085
31  1.093595
32  1.221103
33  3.286744
34  4.833495
35  4.405051
36  2.238449
37  2.970209
